In [1]:
from skyfield.api import Topos, load, Star
from skyfield.data import hipparcos
import numpy as np
import pandas as pd
from astropy.time import Time

In [2]:
# Check which server we're on (in case the data is in different places on different servers)
import socket
hostname = socket.gethostname()

# Get paths to data
if hostname == "blpc1" or hostname == "blpc2":
    data_path = "/datax/scratch/nstieg/"
elif hostname == "cosmic-gpu-1":
    data_path = "/mnt/cosmic-gpu-1/data0/nstiegle/"
else:
    raise Exception("Data path not known")

# Load in coherent data
coherent_full = pd.read_pickle(data_path + "25GHz_higher_coherent_all_columns.pkl")


In [3]:
sources = coherent_full.groupby("source_name")
source_names = list(sources.groups.keys())
first_source = sources.get_group(source_names[0])
first_time_h = first_source.tstart_h.iloc[0]
first_time = first_source.tstart.iloc[0]

In [4]:
ts = load.timescale()
t = ts.from_astropy(Time(first_time, format="mjd"))
print(t.utc_iso())

2023-12-29T03:49:37Z


In [6]:
first_source.source_name.unique()

array(['2535280716217508992'], dtype=object)

In [5]:
print(first_source.ra_hours.unique())
print(first_source.dec_degrees.unique())

[1.28913333]
[0.944188]


In [38]:
# Load satellite TLE data
stations_url = 'https://celestrak.org/NORAD/elements/active.txt' # All active satellites NORAD tracks
satellites = load.tle_file(stations_url)

OSError: cannot download https://celestrak.org/NORAD/elements/active.txt because HTTP Error 404: Not Found

In [ ]:

# Load timescale and set the time of interest
ts = load.timescale()
t = ts.utc(2023, 7, 29, 3, 0)  # Change this to your desired date and time

# Define the target RA/Dec
target_ra = 150.0  # in degrees
target_dec = 30.0  # in degrees
tolerance = 0.5  # degrees, adjust as needed

# Convert target RA/Dec to radians
target_ra_rad = np.radians(target_ra)
target_dec_rad = np.radians(target_dec)

# Function to calculate the angular separation
def angular_separation(ra1, dec1, ra2, dec2):
    return np.arccos(np.sin(dec1) * np.sin(dec2) + np.cos(dec1) * np.cos(dec2) * np.cos(ra1 - ra2))

# Iterate through each satellite and check its position
for satellite in satellites:
    geocentric = satellite.at(t)
    ra, dec, distance = geocentric.radec()
    
    # Calculate the angular separation
    separation = angular_separation(ra.radians, dec.radians, target_ra_rad, target_dec_rad)
    
    # Check if the satellite is within the tolerance
    if np.degrees(separation) < tolerance:
        print(f"Satellite {satellite.name} is near the target RA/Dec at the given time.")
